In [1]:
import os
import autogen
from autogen.cache import Cache

In [2]:
import sys
import os
# Add the parent directory to the system path
parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_directory not in sys.path:
    sys.path.insert(0, parent_directory)

In [5]:
!pip install langchain-chroma -U -q
!pip install sentence-transformers -q

In [3]:
from finrobot.data_source.earnings_calls_src.main_earningsData import get_earnings_all_docs
from finrobot.data_source.earnings_calls_src import earningsData
from finrobot.data_source.filings_src import sec_main as unstructured_sec_main
from finrobot.data_source.earnings_calls_src.main_earningsData import get_earnings_all_docs
from finrobot.data_source.earnings_calls_src import earningsData
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [4]:
from finrobot.functional.ragquery import rag_database_earnings_call
from finrobot.functional.ragquery import rag_database_sec

In [5]:
#ticker = 'GOOG'
ticker = 'NVDA'
year = '2023'
filing_types = ['10-K','10-Q']
include_amends = True

In [6]:
query_database_earnings_call, earnings_call_quarter_vals, quarter_speaker_dict = rag_database_earnings_call(ticker = ticker, year = year)

Earnings Call Q1
Earnings Call Q2
Earnings Call Q3
Earnings Call Q4


In [7]:
global FROM_MARKDOWN
FROM_MARKDOWN = False

In [8]:
query_database_sec, sec_form_names = rag_database_sec(ticker = ticker, year = year,FROM_MARKDOWN = FROM_MARKDOWN)

Started Scraping
Scraped
Started Extracting
Extracted


In [9]:
sec_form_system_msg = ""
#llm_config = {"model":"gpt-4-turbo"}
for sec_form in sec_form_names:
    if sec_form == "10-K":
        sec_form_system_msg+= "10-K for yearly data, "
    elif "10-Q" in sec_form:
        quarter = sec_form[-1]
        sec_form_system_msg+= f"{sec_form} for Q{quarter} data, "
sec_form_system_msg = sec_form_system_msg[:-2]

earnings_call_system_message = ", ".join(earnings_call_quarter_vals)

system_msg = f"""You are a helpful financial assistant and your task is to select the sec_filings or earnings_call or financial_books to best answer the question.
You can use query_database_sec(question,sec_form) by passing question, relevant sec_form names like {sec_form_system_msg}
or you can use query_database_earnings_call(question,quarter) by passing question and relevant quarter names with possible values {earnings_call_system_message}
or you can use query_database_books(question) to get relevant documents from financial textbooks about valuation and investing philosophies. When you are ready to end the coversation, reply TERMINATE"""

In [10]:
from autogen import ConversableAgent
import os


config_list = autogen.config_list_from_json(
    "../OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4-0125-preview"],
    },
)
llm_config = {
    "config_list": config_list,
    "timeout": 120,
    # "temperature": 0 # for debug convenience
    "temperature": 0.5,
}
user_proxy = ConversableAgent(
    name = "Planner Admin",
    system_message=system_msg,
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
)
tool_proxy = ConversableAgent(
  name="Tool Proxy",
  system_message="Analyze the response from user proxy and decide whether the suggested database is suitable "
  ". Answer in simple yes or no",
  llm_config=False,
  # is_termination_msg=lambda msg: "exit" in msg.get("content",""),
  default_auto_reply="Please select the right database.",
  human_input_mode="ALWAYS",
  )

tools_dict = {
        "sec":[query_database_sec,"Tool to query SEC filings database"],
        "earnings_call": [query_database_earnings_call, "Tool to query earnings call transcripts database"],
    }

In [11]:
from autogen import register_function

for tool_name,tool in tools_dict.items():
  register_function(
      tool[0],
      caller=user_proxy,
      executor=tool_proxy,
      name = tool[0].__name__,
      description=tool[1]
  )

In [12]:
input_text = "What is the strategy of Nvidia for artificial intelligence?"
chat_result = user_proxy.initiate_chat(
        recipient=tool_proxy,
        message=input_text,
        max_turns=10
    )

Planner Admin (to Tool Proxy):

What is the strategy of Nvidia for artificial intelligence?

--------------------------------------------------------------------------------


Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Tool Proxy (to Planner Admin):

Please select the right database.

--------------------------------------------------------------------------------
Planner Admin (to Tool Proxy):

***** Suggested tool call (call_myAznQ11sxs0cW4sde6jiIgG): query_database_earnings_call *****
Arguments: 
{"question":"What is the strategy of Nvidia for artificial intelligence?","quarter":"Q3"}
*********************************************************************************************

--------------------------------------------------------------------------------


Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION query_database_earnings_call...


...

Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Tool Proxy (to Planner Admin):

Please select the right database.

--------------------------------------------------------------------------------
Planner Admin (to Tool Proxy):

TERMINATE

--------------------------------------------------------------------------------


Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.


In [13]:
input_text = "What was forward estimates of Nvidia for the year 2023?"
chat_result = user_proxy.initiate_chat(
        recipient=tool_proxy,
        message=input_text,
        max_turns=10
    )

Planner Admin (to Tool Proxy):

What was forward estimates of Nvidia for the year 2023?

--------------------------------------------------------------------------------


Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Tool Proxy (to Planner Admin):

Please select the right database.

--------------------------------------------------------------------------------
Planner Admin (to Tool Proxy):

***** Suggested tool call (call_eHeZHC88aTbZpqwDx2vW2lG3): query_database_sec *****
Arguments: 
{"question":"What was forward estimates of Nvidia for the year 2023?","sec_form_name":"10-K"}
***********************************************************************************

--------------------------------------------------------------------------------


Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Tool Proxy (to Planner Admin):

Tool Proxy (to Planner Admin):

***** Response from calling tool (call_eHeZHC88aTbZpqwDx2vW2lG3) *****
Error: Function query_database_sec not found.
**********************************************************************

--------------------------------------------------------------------------------
Planner Admin (to Tool Proxy):

***** Suggested tool call (call_GyHn4bMKS3JxrVOkf4c9N0fD): query_database_unstructured_sec *****
Arguments: 
{"question":"What was forward estimates of Nvidia for the year 2023?","sec_form_name":"10-K"}
************************************************************************************************

--------------------------------------------------------------------------------


...


>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Tool Proxy (to Planner Admin):

Please select the right database.

--------------------------------------------------------------------------------
Planner Admin (to Tool Proxy):

The right database to query for forward estimates of Google for the year 2023 would be the SEC filings, specifically the 10-K form, which I have already queried.

--------------------------------------------------------------------------------


Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Tool Proxy (to Planner Admin):

Please select the right database.

--------------------------------------------------------------------------------
Planner Admin (to Tool Proxy):

TERMINATE

--------------------------------------------------------------------------------


Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.


In [14]:
input_text = "What are the risk factors that Nvidia faced this year?"
chat_result = user_proxy.initiate_chat(
        recipient=tool_proxy,
        message=input_text,
        max_turns=10
    )

Planner Admin (to Tool Proxy):

What are the risk factors that Nvidia faced this year?

--------------------------------------------------------------------------------


Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Tool Proxy (to Planner Admin):

Please select the right database.

--------------------------------------------------------------------------------
Planner Admin (to Tool Proxy):

***** Suggested tool call (call_0Ha15DmROFeRgWFOqKKAqAfB): query_database_unstructured_sec *****
Arguments: 
{"question":"What are the risk factors that Nvidia faced this year?","sec_form_name":"10-K"}
************************************************************************************************

--------------------------------------------------------------------------------


Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION query_database_unstructured_sec...


...


>>>>>>>> NO HUMAN INPUT RECEIVED.


Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Tool Proxy (to Planner Admin):

Please select the right database.

--------------------------------------------------------------------------------
Planner Admin (to Tool Proxy):

TERMINATE

--------------------------------------------------------------------------------


Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.
